# This notebook downloads and prepares all election data

In [1]:
import requests

import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

import xlrd
import xlwt

In [2]:
#r = requests.get('https://transition.fec.gov/pubrec/fe2016/federalelections2016.xlsx')

#with open("../raw-data/federal-election-results/federalelections2016.xlsx",'wb') as f: 
  
    # Saving received content as a png file in 
    # binary format 
  
    # write the contents of the response (r.content) 
    # to a new file in binary mode. 
 #   f.write(r.content) 

In [3]:
house_by_dist = pd.read_excel("../raw-data/federal-election-results/federalelections2016.xlsx", sheet_name='2016 US House Results by State')
house_by_dist.head(50)

,1,STATE ABBREVIATION,STATE,D,FEC ID#,(I),CANDIDATE NAME (First),CANDIDATE NAME (Last),CANDIDATE NAME,TOTAL VOTES,...,RUNOFF VOTES,RUNOFF %,GENERAL VOTES,GENERAL %,GE RUNOFF ELECTION VOTES (LA),GE RUNOFF ELECTION % (LA),"COMBINED GE PARTY TOTALS (CT, NY, SC)","COMBINED % (CT, NY, SC)",GE WINNER INDICATOR,FOOTNOTES
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,AL,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,AL,Alabama,01,H4AL01123,(I),Bradley,Byrne,"Byrne, Bradley",NaN,...,NaN,NaN,208083,0.963825,NaN,NaN,NaN,NaN,W,NaN
3,5,AL,Alabama,01,H6AL01060,NaN,Dean,Young,"Young, Dean",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,AL,Alabama,01,NaN,NaN,NaN,NaN,NaN,Party Votes:,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,AL,Alabama,01,NaN,NaN,NaN,Scattered,Scattered,NaN,...,NaN,NaN,7810,0.036175,NaN,NaN,NaN,NaN,NaN,NaN
6,8,AL,Alabama,01,NaN,NaN,NaN,NaN,NaN,District Votes:,...,NaN,NaN,215893,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9,AL,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10,AL,Alabama,02,H0AL02087,(I),Martha,Roby,"Roby, Martha",NaN,...,NaN,NaN,134886,0.487685,NaN,NaN,NaN,NaN,W,NaN
9,11,AL,Alabama,02,H6AL02142,NaN,Becky,Gerritson,"Gerritson, Becky",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
house_by_dist["D_ID"] = house_by_dist["STATE ABBREVIATION"] + "_" + house_by_dist["D"].map(str)

house_by_dist.rename(columns={1:'row_excel',
                              'TOTAL VOTES':'TOTAL_VOTES',
                              'GENERAL VOTES ':'GENERAL_VOTES',
                              'STATE ABBREVIATION':'STATE_ABBREVIATION'}
                              , inplace=True)
list(house_by_dist.columns.values)



['row_excel',
 'STATE_ABBREVIATION',
 'STATE',
 'D',
 'FEC ID#',
 '(I)',
 'CANDIDATE NAME (First)',
 'CANDIDATE NAME (Last)',
 'CANDIDATE NAME',
 'TOTAL_VOTES',
 'PARTY',
 'PRIMARY VOTES',
 'PRIMARY %',
 'RUNOFF VOTES',
 'RUNOFF %',
 'GENERAL_VOTES',
 'GENERAL %',
 'GE RUNOFF ELECTION VOTES (LA)',
 'GE RUNOFF ELECTION % (LA)',
 'COMBINED GE PARTY TOTALS (CT, NY, SC)',
 'COMBINED % (CT, NY, SC)',
 'GE WINNER INDICATOR',
 'FOOTNOTES',
 'D_ID']

In [5]:
house_by_dist = house_by_dist[['row_excel','D_ID', 'PARTY', 'TOTAL_VOTES','GENERAL_VOTES', "STATE_ABBREVIATION", "D"]]

house_by_dist.head()

,row_excel,D_ID,PARTY,TOTAL_VOTES,GENERAL_VOTES,STATE_ABBREVIATION,D
0,2,NaN,NaN,NaN,NaN,NaN,NaN
1,3,AL_nan,NaN,NaN,NaN,AL,NaN
2,4,AL_01,R,NaN,208083,AL,01
3,5,AL_01,R,NaN,NaN,AL,01
4,6,AL_01,R,Party Votes:,NaN,AL,01


In [6]:
print(house_by_dist.STATE_ABBREVIATION.unique())

[nan 'AL' 'AK' 'AS' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'GU' 'HI'
 'ID' 'IL' 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO'
 'MT' 'NE' 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'MP' 'OH' 'OK' 'OR' 'PA'
 'PR' 'RI' 'SC' 'SD' 'TN' 'TX' 'UT' 'VT' 'VA' 'VI' 'WA' 'WV' 'WI' 'WY']


In [7]:
state_nan = house_by_dist[(house_by_dist.STATE_ABBREVIATION.isnull())]
state_nan.head()
#state_nan.head(500)

,row_excel,D_ID,PARTY,TOTAL_VOTES,GENERAL_VOTES,STATE_ABBREVIATION,D
0,2,NaN,NaN,NaN,NaN,NaN,NaN
42,44,NaN,NaN,NaN,NaN,NaN,NaN
45,47,NaN,NaN,NaN,NaN,NaN,NaN
64,66,NaN,NaN,NaN,NaN,NaN,NaN
67,69,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(house_by_dist.count())
house_by_dist = house_by_dist.dropna(subset=['STATE_ABBREVIATION'])
print(house_by_dist.count())

row_excel             4131
D_ID                  4069
PARTY                 3053
TOTAL_VOTES           1120
GENERAL_VOTES         1964
STATE_ABBREVIATION    4069
D                     3971
dtype: int64
row_excel             4069
D_ID                  4069
PARTY                 3053
TOTAL_VOTES           1120
GENERAL_VOTES         1964
STATE_ABBREVIATION    4069
D                     3971
dtype: int64


In [9]:
print(house_by_dist.D.unique())

[nan '01' '02' '03' '04' '05' '06' '07' 'H' '00' '08' '09' '10' '11' '12'
 '13' '14' '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26'
 '27' '28' '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40'
 '41' '42' '43' '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '04 '
 '01 - FULL TERM' '01 - UNEXPIRED TERM' '1 - UNEXPIRED TERM' 10 11 12 13
 14 15 16 17 18 19 20 21 22 23 24 25 26 27 '02 - FULL TERM'
 '02 - UNEXPIRED TERM']


In [10]:
D_nan = house_by_dist[(house_by_dist.D.isnull())]
D_nan.head()
#D_nan.head(500)

,row_excel,D_ID,PARTY,TOTAL_VOTES,GENERAL_VOTES,STATE_ABBREVIATION,D
1,3,AL_nan,NaN,NaN,NaN,AL,NaN
7,9,AL_nan,NaN,NaN,NaN,AL,NaN
15,17,AL_nan,NaN,NaN,NaN,AL,NaN
22,24,AL_nan,NaN,NaN,NaN,AL,NaN
28,30,AL_nan,NaN,NaN,NaN,AL,NaN


In [11]:
print(house_by_dist.count())
house_by_dist = house_by_dist.dropna(subset=['D'])
print(house_by_dist.count())

row_excel             4069
D_ID                  4069
PARTY                 3053
TOTAL_VOTES           1120
GENERAL_VOTES         1964
STATE_ABBREVIATION    4069
D                     3971
dtype: int64
row_excel             3971
D_ID                  3971
PARTY                 3050
TOTAL_VOTES           1115
GENERAL_VOTES         1962
STATE_ABBREVIATION    3971
D                     3971
dtype: int64


In [12]:
# H is a row with state total votes
D_H = house_by_dist[(house_by_dist.D == "H")]
D_H.head()
#D_H.head(500)

,row_excel,D_ID,PARTY,TOTAL_VOTES,GENERAL_VOTES,STATE_ABBREVIATION,D
43,45,AL_H,NaN,Total State Votes:,1889685,AL,H
65,67,AK_H,NaN,Total State Votes:,308198,AK,H
74,76,AS_H,NaN,Total State Votes:,11834,AS,H
178,180,AZ_H,NaN,Total State Votes:,2412064,AZ,H
202,204,AR_H,NaN,Total State Votes:,1068577,AR,H


In [13]:
print(house_by_dist.count())
house_by_dist = house_by_dist[(house_by_dist["D"] != "H")]
print(house_by_dist.count())

row_excel             3971
D_ID                  3971
PARTY                 3050
TOTAL_VOTES           1115
GENERAL_VOTES         1962
STATE_ABBREVIATION    3971
D                     3971
dtype: int64
row_excel             3915
D_ID                  3915
PARTY                 3050
TOTAL_VOTES           1059
GENERAL_VOTES         1906
STATE_ABBREVIATION    3915
D                     3915
dtype: int64


In [14]:
# Remove D and State Abbreviation, use only D_ID form here
house_by_dist = house_by_dist.drop(['D','STATE_ABBREVIATION'], axis=1)


In [15]:
# Other aggregate rows. Party aggregates were only applicable in primary elections
votes_party_agg = house_by_dist[(house_by_dist.TOTAL_VOTES == "Party Votes:")]
votes_party_agg.head()
#votes_party_agg.head(500)

,row_excel,D_ID,PARTY,TOTAL_VOTES,GENERAL_VOTES
4,6,AL_01,R,Party Votes:,NaN
11,13,AL_02,R,Party Votes:,NaN
18,20,AL_03,R,Party Votes:,NaN
25,27,AL_04,R,Party Votes:,NaN
51,53,AK_00,R,Party Votes:,NaN


In [16]:
print(house_by_dist.count())
house_by_dist = house_by_dist[(house_by_dist.TOTAL_VOTES != "Party Votes:")]
print(house_by_dist.count())

row_excel        3915
D_ID             3915
PARTY            3050
TOTAL_VOTES      1059
GENERAL_VOTES    1906
dtype: int64
row_excel        3298
D_ID             3298
PARTY            2434
TOTAL_VOTES       442
GENERAL_VOTES    1906
dtype: int64


In [17]:
# Other aggregate rows. PDistrict votes showing the total number of voters in that district
votes_dist_agg = house_by_dist[(house_by_dist.TOTAL_VOTES == "District Votes:")]
votes_dist_agg.head()
#votes_dist_agg.head(500)

,row_excel,D_ID,PARTY,TOTAL_VOTES,GENERAL_VOTES
6,8,AL_01,NaN,District Votes:,215893
14,16,AL_02,NaN,District Votes:,276584
21,23,AL_03,NaN,District Votes:,287104
27,29,AL_04,NaN,District Votes:,239444
32,34,AL_05,NaN,District Votes:,308326


In [18]:
print(house_by_dist.count())
house_by_dist = house_by_dist[(house_by_dist.TOTAL_VOTES != "District Votes:")]
print(house_by_dist.count())

row_excel        3298
D_ID             3298
PARTY            2434
TOTAL_VOTES       442
GENERAL_VOTES    1906
dtype: int64
row_excel        2856
D_ID             2856
PARTY            2433
TOTAL_VOTES         0
GENERAL_VOTES    1466
dtype: int64


In [19]:
# Remove TOTAL_VOTES
house_by_dist = house_by_dist.drop(['TOTAL_VOTES'], axis=1)


In [20]:
votes_dist_agg.rename(columns={'GENERAL_VOTES':'DISTRICT_VOTES'}, inplace=True)
votes_dist_agg = votes_dist_agg[['D_ID','DISTRICT_VOTES']]
votes_dist_agg.head()

,D_ID,DISTRICT_VOTES
6,AL_01,215893
14,AL_02,276584
21,AL_03,287104
27,AL_04,239444
32,AL_05,308326


In [21]:
house_by_dist = pd.merge(house_by_dist, votes_dist_agg, on='D_ID')

In [22]:
print(house_by_dist.PARTY.unique())

['R' 'W' 'D' 'LIB' 'NAF' 'IND' nan 'W(GRE)/GRE' 'W(LIB)' 'W(GRE)' 'W(D)'
 'NOP' 'GRE' 'W(R)/R' 'PAF' 'W(NOP)' 'LIB  ' 'WF' 'IP' 'R/W' 'DCG'
 'W(DCG)' 'W(R)' 'LBF' 'NPA' 'R   ' 'R  ' 'W(IP)' 'N' 'CON' 'NNE' 'R\xa0'
 'D\xa0' 'OTH' 'W(IND)' 'U' 'UST' 'W(D)/D' 'NLP' 'WC' 'W ' 'DFL' 'IDP'
 'LMN' 'REF' 'VPA' 'NPY' 'IAP' 'WDB' 'AO' 'MGW' 'RNN' 'PIP' 'FPR' 'EG'
 'WUA' 'NSA' 'WOP' 'NBP' 'FI' 'LMP' 'TED' 'WTP' 'CRV' 'WEP' 'R/TRP' 'BLM'
 'HBP' 'SID' 'TGP' 'PCC' 'UPJ' 'DNL' 'W(DNL)' 'D/IP' 'R/IP' 'IP/R' 'PRO'
 'D/PRO/WF/IP' 'R/CON' 'PG' 'W(D)/W' 'AM' 'UN' 'D/R' 'LBU' 'W(PRO)' 'INP'
 'WRN' 'TC' 'W(WG)' 'W(CON)']


In [23]:
# Empty parties is at this point just empty rows
PARTY_nan = house_by_dist[(house_by_dist.PARTY.isnull())]
PARTY_nan.head()
#PARTY_nan.head(500)

,row_excel,D_ID,PARTY,GENERAL_VOTES,DISTRICT_VOTES
41,77,AS_00,NaN,NaN,11834
113,215,CA_01,NaN,NaN,314036
119,222,CA_02,NaN,NaN,330766
123,227,CA_03,NaN,NaN,256966
127,232,CA_04,NaN,NaN,350978


In [24]:
print(house_by_dist.count())
house_by_dist = house_by_dist.dropna(subset=['PARTY'])
print(house_by_dist.count())

row_excel         2825
D_ID              2825
PARTY             2404
GENERAL_VOTES     1454
DISTRICT_VOTES    2818
dtype: int64
row_excel         2404
D_ID              2404
PARTY             2404
GENERAL_VOTES     1454
DISTRICT_VOTES    2399
dtype: int64


In [25]:
# Remove excessive spaces , for example turn " R" into "R", otherwise they are considered different
house_by_dist.PARTY = house_by_dist.PARTY.str.strip()

#Parties to not consider
house_by_dist["PARTY_clean"] = house_by_dist.PARTY


In [26]:
house_by_dist.loc[house_by_dist.PARTY == "W(GRE)"    , 'PARTY_clean'] = "GRE"
house_by_dist.loc[house_by_dist.PARTY == "W(GRE)/GRE", 'PARTY_clean'] = "GRE"

house_by_dist.loc[house_by_dist.PARTY == "W(LIB)"    , 'PARTY_clean'] = "LIB"

house_by_dist.loc[house_by_dist.PARTY == "W(D)"      , 'PARTY_clean'] = "D"
house_by_dist.loc[house_by_dist.PARTY == "W(D)/D"    , 'PARTY_clean'] = "D"
house_by_dist.loc[house_by_dist.PARTY == "D\xa0"     , 'PARTY_clean'] = "D"

house_by_dist.loc[house_by_dist.PARTY == "W(R)"      , 'PARTY_clean'] = "R"
house_by_dist.loc[house_by_dist.PARTY == "W(R)/R"    , 'PARTY_clean'] = "R"
house_by_dist.loc[house_by_dist.PARTY == "R\xa0"     , 'PARTY_clean'] = "R"

house_by_dist.loc[house_by_dist.PARTY == "W(DNL)"    , 'PARTY_clean'] = "DNL"

house_by_dist.loc[house_by_dist.PARTY == "W(PRO)"    , 'PARTY_clean'] = "PRO"

house_by_dist.loc[house_by_dist.PARTY == "W(WG)"     , 'PARTY_clean'] = "WG"

house_by_dist.loc[house_by_dist.PARTY == "W(IP)"     , 'PARTY_clean'] = "IP"

house_by_dist.loc[house_by_dist.PARTY == "W(DCG)"    , 'PARTY_clean'] = "DCG"

house_by_dist.loc[house_by_dist.PARTY == "W(CON)"    , 'PARTY_clean'] = "CON"

house_by_dist.loc[house_by_dist.PARTY == "W(NPP)"    , 'PARTY_clean'] = "NPP"

house_by_dist.loc[house_by_dist.PARTY == "W(PPD)"    , 'PARTY_clean'] = "PPD"


print(sorted(house_by_dist.PARTY_clean.unique()))

['AM', 'AO', 'BLM', 'CON', 'CRV', 'D', 'D/IP', 'D/PRO/WF/IP', 'D/R', 'DCG', 'DFL', 'DNL', 'EG', 'FI', 'FPR', 'GRE', 'HBP', 'IAP', 'IDP', 'IND', 'INP', 'IP', 'IP/R', 'LBF', 'LBU', 'LIB', 'LMN', 'LMP', 'MGW', 'N', 'NAF', 'NBP', 'NLP', 'NNE', 'NOP', 'NPA', 'NPY', 'NSA', 'OTH', 'PAF', 'PCC', 'PG', 'PIP', 'PRO', 'R', 'R/CON', 'R/IP', 'R/TRP', 'R/W', 'REF', 'RNN', 'SID', 'TC', 'TED', 'TGP', 'U', 'UN', 'UPJ', 'UST', 'VPA', 'W', 'W(D)/W', 'W(IND)', 'W(NOP)', 'WC', 'WDB', 'WEP', 'WF', 'WG', 'WOP', 'WRN', 'WTP', 'WUA']


In [27]:


# NOP - No Party Affiliance
house_by_dist.loc[house_by_dist.PARTY == "NOP"    , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "W(NOP)" , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "NAF"    , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "NNE"    , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "NPA"    , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "NPY"    , 'PARTY_clean'] = "no-party"

# OTH - Other
house_by_dist.loc[house_by_dist.PARTY == "OTH"    , 'PARTY_clean'] = "no-party"
# IND independent
house_by_dist.loc[house_by_dist.PARTY == "IND"    , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "INP"    , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "W(IND)" , 'PARTY_clean'] = "no-party"

#Ambigious
house_by_dist.loc[house_by_dist.PARTY == "D/IP"        , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "D/PRO/WF/IP" , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "D/R"         , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "IP/R"        , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "R/CON"       , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "R/IP"        , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "R/TRP"       , 'PARTY_clean'] = "no-party"

# Un enrolloed
house_by_dist.loc[house_by_dist.PARTY == "U"      , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "UN"     , 'PARTY_clean'] = "no-party"

#Write ins
# W - unspecified write ins
house_by_dist.loc[house_by_dist.PARTY == "W"      , 'PARTY_clean'] = "no-party"
# Ambigious write ins 
#  - W(D)/W, R/W
house_by_dist.loc[house_by_dist.PARTY == "W(D)/W" , 'PARTY_clean'] = "no-party"
house_by_dist.loc[house_by_dist.PARTY == "R/W"    , 'PARTY_clean'] = "no-party"

print(sorted(house_by_dist.PARTY_clean.unique()))

['AM', 'AO', 'BLM', 'CON', 'CRV', 'D', 'DCG', 'DFL', 'DNL', 'EG', 'FI', 'FPR', 'GRE', 'HBP', 'IAP', 'IDP', 'IP', 'LBF', 'LBU', 'LIB', 'LMN', 'LMP', 'MGW', 'N', 'NBP', 'NLP', 'NSA', 'PAF', 'PCC', 'PG', 'PIP', 'PRO', 'R', 'REF', 'RNN', 'SID', 'TC', 'TED', 'TGP', 'UPJ', 'UST', 'VPA', 'WC', 'WDB', 'WEP', 'WF', 'WG', 'WOP', 'WRN', 'WTP', 'WUA', 'no-party']


In [28]:
votes_nan = house_by_dist[(house_by_dist.GENERAL_VOTES.isnull())]
votes_nan.head()
#votes_nan.head(500)

,row_excel,D_ID,PARTY,GENERAL_VOTES,DISTRICT_VOTES,PARTY_clean
1,5,AL_01,R,NaN,215893,R
4,11,AL_02,R,NaN,276584,R
5,12,AL_02,R,NaN,276584,R
9,19,AL_03,R,NaN,287104,R
13,26,AL_04,R,NaN,239444,R


In [29]:
print(house_by_dist.count())
house_by_dist = house_by_dist.dropna(subset=['GENERAL_VOTES'])
print(house_by_dist.count())

row_excel         2404
D_ID              2404
PARTY             2404
GENERAL_VOTES     1454
DISTRICT_VOTES    2399
PARTY_clean       2404
dtype: int64
row_excel         1454
D_ID              1454
PARTY             1454
GENERAL_VOTES     1454
DISTRICT_VOTES    1452
PARTY_clean       1454
dtype: int64


In [30]:
house_by_dist.loc[house_by_dist.D_ID == "AR_02"].head(100)

,row_excel,D_ID,PARTY,GENERAL_VOTES,DISTRICT_VOTES,PARTY_clean
97,188,AR_02,R,176472,302464,R
99,191,AR_02,D,111347,302464,D
100,192,AR_02,LIB,14342,302464,LIB
101,193,AR_02,W,303,302464,no-party


In [31]:
control_sum_series = house_by_dist['GENERAL_VOTES'].groupby([house_by_dist.D_ID]).sum()

control_sum = pd.DataFrame({'D_ID':control_sum_series.index, 
                            'DISTRICT_VOTES_ctrl':control_sum_series.values})
control_sum.head()

,D_ID,DISTRICT_VOTES_ctrl
0,AK_00,308198
1,AL_01,215893
2,AL_02,276584
3,AL_03,287104
4,AL_04,239444


In [32]:
house_by_dist = pd.merge(house_by_dist, control_sum, on='D_ID')

In [33]:
#List observation where control sum is incorrect
house_by_dist[house_by_dist.DISTRICT_VOTES != house_by_dist.DISTRICT_VOTES_ctrl].head(100)

,row_excel,D_ID,PARTY,GENERAL_VOTES,DISTRICT_VOTES,PARTY_clean,DISTRICT_VOTES_ctrl
242,694,FL_04,D,113088,409662,D,122153
243,695,FL_04,NPA,9054,409662,no-party,122153
244,696,FL_04,W,11,409662,no-party,122153
294,833,FL_24,D,Unopposed,NaN,D,Unopposed
454,1384,KY_1 - UNEXPIRED TERM,D,80813,290623,D,80813
1051,2894,OK_01,R,Unopposed,NaN,R,Unopposed
1180,3284,SC_05,R,175909,297781,R,291542
1181,3287,SC_05,D,115437,297781,D,291542
1182,3290,SC_05,W,196,297781,no-party,291542
1183,3288,SC_06,AM,6239,253901,AM,260140


In [34]:
house_by_dist.head(100)

,row_excel,D_ID,PARTY,GENERAL_VOTES,DISTRICT_VOTES,PARTY_clean,DISTRICT_VOTES_ctrl
0,4,AL_01,R,208083,215893,R,215893
1,7,AL_01,W,7810,215893,no-party,215893
2,10,AL_02,R,134886,276584,R,276584
3,14,AL_02,D,112089,276584,D,276584
4,15,AL_02,W,29609,276584,no-party,276584
5,18,AL_03,R,192164,287104,R,287104
6,21,AL_03,D,94549,287104,D,287104
7,22,AL_03,W,391,287104,no-party,287104
8,25,AL_04,R,235925,239444,R,239444
9,28,AL_04,W,3519,239444,no-party,239444


In [35]:
list(house_by_dist.columns.values)

['row_excel',
 'D_ID',
 'PARTY',
 'GENERAL_VOTES',
 'DISTRICT_VOTES',
 'PARTY_clean',
 'DISTRICT_VOTES_ctrl']